### Deliverable 3 : Optimize the Model

In [1]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler,OneHotEncoder
import pandas as pd
import tensorflow as tf

#  Import and read the charity_data.csv.
import pandas as pd 
application_df = pd.read_csv("Resources/charity_data.csv")
application_df.head()

EIN                                      NAME APPLICATION_TYPE  \
0  10520599              BLUE KNIGHTS MOTORCYCLE CLUB              T10   
1  10531628    AMERICAN CHESAPEAKE CLUB CHARITABLE TR               T3   
2  10547893        ST CLOUD PROFESSIONAL FIREFIGHTERS               T5   
3  10553066            SOUTHSIDE ATHLETIC ASSOCIATION               T3   
4  10556103  GENETIC RESEARCH INSTITUTE OF THE DESERT               T3   

        AFFILIATION CLASSIFICATION      USE_CASE  ORGANIZATION  STATUS  \
0       Independent          C1000    ProductDev   Association       1   
1       Independent          C2000  Preservation  Co-operative       1   
2  CompanySponsored          C3000    ProductDev   Association       1   
3  CompanySponsored          C2000  Preservation         Trust       1   
4       Independent          C1000     Heathcare         Trust       1   

      INCOME_AMT SPECIAL_CONSIDERATIONS  ASK_AMT  IS_SUCCESSFUL  
0              0                      N     5000              1  
1         1-9999                      N   108590              1  
2              0                      N     5000              0  
3    10000-24999                      N     6692              1  
4  100000-499999                      N   142590              1

In [2]:
# Drop the non-beneficial ID columns, 'EIN' and 'NAME'.
application_df = application_df.drop(columns=["EIN", "NAME"], axis=1)
application_df.head()

APPLICATION_TYPE       AFFILIATION CLASSIFICATION      USE_CASE  \
0              T10       Independent          C1000    ProductDev   
1               T3       Independent          C2000  Preservation   
2               T5  CompanySponsored          C3000    ProductDev   
3               T3  CompanySponsored          C2000  Preservation   
4               T3       Independent          C1000     Heathcare   

   ORGANIZATION  STATUS     INCOME_AMT SPECIAL_CONSIDERATIONS  ASK_AMT  \
0   Association       1              0                      N     5000   
1  Co-operative       1         1-9999                      N   108590   
2   Association       1              0                      N     5000   
3         Trust       1    10000-24999                      N     6692   
4         Trust       1  100000-499999                      N   142590   

   IS_SUCCESSFUL  
0              1  
1              1  
2              0  
3              1  
4              1

In [3]:
# Determine the number of unique values in each column.
application_df.nunique()

APPLICATION_TYPE            17
AFFILIATION                  6
CLASSIFICATION              71
USE_CASE                     5
ORGANIZATION                 4
STATUS                       2
INCOME_AMT                   9
SPECIAL_CONSIDERATIONS       2
ASK_AMT                   8747
IS_SUCCESSFUL                2
dtype: int64

In [4]:
# APPLICATION_TYPE value counts for binning
application_counts = application_df['APPLICATION_TYPE'].value_counts()

# Determine which values to replace if counts are less than ...?
replace_application = list(application_counts[application_counts < 500].index)

# Replace in dataframe
for app in replace_application:
    application_df.APPLICATION_TYPE = application_df.APPLICATION_TYPE.replace(app,"Other")

In [5]:
# CLASSIFICATION value counts for binning
classification_value_counts = application_df['CLASSIFICATION'].value_counts()

# Determine which values to replace if counts are less than ..?
replace_class = list(classification_value_counts[classification_value_counts < 1000].index)

# Replace in dataframe
for cls in replace_class:
    application_df.CLASSIFICATION = application_df.CLASSIFICATION.replace(cls,"Other")

In [6]:
# Look at ASK_AMT value counts for binning
ask_amt_counts = application_df.ASK_AMT.value_counts()
ask_amt_counts

5000        25398
10478           3
15583           3
63981           3
6725            3
            ...  
5371754         1
30060           1
43091152        1
18683           1
36500179        1
Name: ASK_AMT, Length: 8747, dtype: int64

In [7]:
# Generate our categorical variable lists
application_cat = ["APPLICATION_TYPE", "AFFILIATION", "CLASSIFICATION", "USE_CASE", "ORGANIZATION", "INCOME_AMT", 
                  "SPECIAL_CONSIDERATIONS"]

In [8]:
# Create a OneHotEncoder instance
enc = OneHotEncoder(sparse=False)

# Fit and transform the OneHotEncoder using the categorical variable list
encode_df = pd.DataFrame(enc.fit_transform(application_df[application_cat]))

# Add the encoded variable names to the dataframe
encode_df.columns = enc.get_feature_names(application_cat)
encode_df.head()

/opt/anaconda3/envs/tf2/lib/python3.7/site-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


APPLICATION_TYPE_Other  APPLICATION_TYPE_T10  APPLICATION_TYPE_T19  \
0                     0.0                   1.0                   0.0   
1                     0.0                   0.0                   0.0   
2                     0.0                   0.0                   0.0   
3                     0.0                   0.0                   0.0   
4                     0.0                   0.0                   0.0   

   APPLICATION_TYPE_T3  APPLICATION_TYPE_T4  APPLICATION_TYPE_T5  \
0                  0.0                  0.0                  0.0   
1                  1.0                  0.0                  0.0   
2                  0.0                  0.0                  1.0   
3                  1.0                  0.0                  0.0   
4                  1.0                  0.0                  0.0   

   APPLICATION_TYPE_T6  APPLICATION_TYPE_T7  APPLICATION_TYPE_T8  \
0                  0.0                  0.0                  0.0   
1                  0.0                  0.0                  0.0   
2                  0.0                  0.0                  0.0   
3                  0.0                  0.0                  0.0   
4                  0.0                  0.0                  0.0   

   AFFILIATION_CompanySponsored  ...  INCOME_AMT_1-9999  \
0                           0.0  ...                0.0   
1                           0.0  ...                1.0   
2                           1.0  ...                0.0   
3                           1.0  ...                0.0   
4                           0.0  ...                0.0   

   INCOME_AMT_10000-24999  INCOME_AMT_100000-499999  INCOME_AMT_10M-50M  \
0                     0.0                       0.0                 0.0   
1                     0.0                       0.0                 0.0   
2                     0.0                       0.0                 0.0   
3                     1.0                       0.0                 0.0   
4                     0.0                       1.0                 0.0   

   INCOME_AMT_1M-5M  INCOME_AMT_25000-99999  INCOME_AMT_50M+  \
0               0.0                     0.0              0.0   
1               0.0                     0.0              0.0   
2               0.0                     0.0              0.0   
3               0.0                     0.0              0.0   
4               0.0                     0.0              0.0   

   INCOME_AMT_5M-10M  SPECIAL_CONSIDERATIONS_N  SPECIAL_CONSIDERATIONS_Y  
0                0.0                       1.0                       0.0  
1                0.0                       1.0                       0.0  
2                0.0                       1.0                       0.0  
3                0.0                       1.0                       0.0  
4                0.0                       1.0                       0.0  

[5 rows x 41 columns]

In [9]:
# Merge one-hot encoded features and drop the originals
application_df = application_df.merge(encode_df, left_index=True, right_index=True)
application_df = application_df.drop(columns=application_cat, axis=1)

In [10]:
# Split our preprocessed data into our features and target arrays
y = application_df['IS_SUCCESSFUL'].values
X = application_df.drop(['IS_SUCCESSFUL'], 1).values

# Split the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=78)

/opt/anaconda3/envs/tf2/lib/python3.7/site-packages/ipykernel_launcher.py:3: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  This is separate from the ipykernel package so we can avoid doing imports until


In [11]:
# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

### Adding more Neurons to a Hidden Layer

In [12]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
number_input_features = len(X_train_scaled[0])
hidden_nodes_layer1 = 100
hidden_nodes_layer2 = 30


nn = tf.keras.models.Sequential()

# First hidden layer
nn.add(
    tf.keras.layers.Dense(units=hidden_nodes_layer1, input_dim=number_input_features, activation="relu")
)

# Second hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer2, activation="relu"))

# Output layer
nn.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Check the structure of the model
nn.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 100)               4400      
_________________________________________________________________
dense_1 (Dense)              (None, 30)                3030      
_________________________________________________________________
dense_2 (Dense)              (None, 1)                 31        
Total params: 7,461
Trainable params: 7,461
Non-trainable params: 0
_________________________________________________________________


2022-04-30 16:27:04.816350: I tensorflow/core/platform/cpu_feature_guard.cc:145] This TensorFlow binary is optimized with Intel(R) MKL-DNN to use the following CPU instructions in performance critical operations:  SSE4.1 SSE4.2
To enable them in non-MKL-DNN operations, rebuild TensorFlow with the appropriate compiler flags.
2022-04-30 16:27:04.816816: I tensorflow/core/common_runtime/process_util.cc:115] Creating new thread pool with default inter op setting: 8. Tune using inter_op_parallelism_threads for best performance.


In [13]:
# Compile the model
nn.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

In [14]:
# Import checkpoint dependencies
import os
from tensorflow.keras.callbacks import ModelCheckpoint

# Define the checkpoint path and filenames
os.makedirs("checkpoints/",exist_ok=True)
checkpoint_path = "checkpoints/weights.{epoch:02d}.hdf5"

# Compile the model
nn.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

# Create a callback that saves the model's weights every epoch
cp_callback = ModelCheckpoint(
    filepath=checkpoint_path,
    verbose=1,
    save_weights_only=True,
    save_freq='epoch')

# Train the model
fit_model = nn.fit(X_train_scaled,y_train,epochs=100,callbacks=[cp_callback])

Train on 25724 samples
Epoch 1/100
25440/25724 [============================>.] - ETA: 0s - loss: 0.5695 - accuracy: 0.7221
Epoch 00001: saving model to checkpoints/weights.01.hdf5
25724/25724 [==============================] - 4s 172us/sample - loss: 0.5696 - accuracy: 0.7221
Epoch 2/100
25568/25724 [============================>.] - ETA: 0s - loss: 0.5551 - accuracy: 0.7293
Epoch 00002: saving model to checkpoints/weights.02.hdf5
25724/25724 [==============================] - 3s 127us/sample - loss: 0.5554 - accuracy: 0.7292
Epoch 3/100
25248/25724 [============================>.] - ETA: 0s - loss: 0.5520 - accuracy: 0.7303
Epoch 00003: saving model to checkpoints/weights.03.hdf5
25724/25724 [==============================] - 3s 115us/sample - loss: 0.5522 - accuracy: 0.7301
Epoch 4/100
25696/25724 [============================>.] - ETA: 0s - loss: 0.5513 - accuracy: 0.7321
Epoch 00004: saving model to checkpoints/weights.04.hdf5
25724/25724 [==============================] - 3s 115u

Epoch 33/100
25696/25724 [============================>.] - ETA: 0s - loss: 0.5413 - accuracy: 0.7381
Epoch 00033: saving model to checkpoints/weights.33.hdf5
25724/25724 [==============================] - 3s 113us/sample - loss: 0.5411 - accuracy: 0.7382
Epoch 34/100
25440/25724 [============================>.] - ETA: 0s - loss: 0.5402 - accuracy: 0.7399
Epoch 00034: saving model to checkpoints/weights.34.hdf5
25724/25724 [==============================] - 3s 114us/sample - loss: 0.5402 - accuracy: 0.7399
Epoch 35/100
25568/25724 [============================>.] - ETA: 0s - loss: 0.5401 - accuracy: 0.7387
Epoch 00035: saving model to checkpoints/weights.35.hdf5
25724/25724 [==============================] - 3s 125us/sample - loss: 0.5403 - accuracy: 0.7385
Epoch 36/100
25696/25724 [============================>.] - ETA: 0s - loss: 0.5395 - accuracy: 0.7391
Epoch 00036: saving model to checkpoints/weights.36.hdf5
25724/25724 [==============================] - 3s 112us/sample - loss: 0.

25568/25724 [============================>.] - ETA: 0s - loss: 0.5369 - accuracy: 0.7412
Epoch 00065: saving model to checkpoints/weights.65.hdf5
25724/25724 [==============================] - 3s 113us/sample - loss: 0.5372 - accuracy: 0.7406
Epoch 66/100
25472/25724 [============================>.] - ETA: 0s - loss: 0.5373 - accuracy: 0.7404
Epoch 00066: saving model to checkpoints/weights.66.hdf5
25724/25724 [==============================] - 3s 118us/sample - loss: 0.5374 - accuracy: 0.7402
Epoch 67/100
25280/25724 [============================>.] - ETA: 0s - loss: 0.5373 - accuracy: 0.7398
Epoch 00067: saving model to checkpoints/weights.67.hdf5
25724/25724 [==============================] - 3s 113us/sample - loss: 0.5373 - accuracy: 0.7400
Epoch 68/100
25440/25724 [============================>.] - ETA: 0s - loss: 0.5363 - accuracy: 0.7402
Epoch 00068: saving model to checkpoints/weights.68.hdf5
25724/25724 [==============================] - 3s 116us/sample - loss: 0.5367 - accura

25632/25724 [============================>.] - ETA: 0s - loss: 0.5360 - accuracy: 0.7406
Epoch 00097: saving model to checkpoints/weights.97.hdf5
25724/25724 [==============================] - 11s 442us/sample - loss: 0.5357 - accuracy: 0.7409
Epoch 98/100
25472/25724 [============================>.] - ETA: 0s - loss: 0.5353 - accuracy: 0.7417
Epoch 00098: saving model to checkpoints/weights.98.hdf5
25724/25724 [==============================] - 14s 537us/sample - loss: 0.5357 - accuracy: 0.7414
Epoch 99/100
25632/25724 [============================>.] - ETA: 0s - loss: 0.5354 - accuracy: 0.7413
Epoch 00099: saving model to checkpoints/weights.99.hdf5
25724/25724 [==============================] - 6s 238us/sample - loss: 0.5353 - accuracy: 0.7414
Epoch 100/100
25664/25724 [============================>.] - ETA: 0s - loss: 0.5356 - accuracy: 0.7415
Epoch 00100: saving model to checkpoints/weights.100.hdf5
25724/25724 [==============================] - 12s 465us/sample - loss: 0.5357 - a

In [15]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

8575/1 - 1s - loss: 0.5498 - accuracy: 0.7264
Loss: 0.5584584420832531, Accuracy: 0.7264139652252197


### Adding more Hidden Layers

In [16]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
number_input_features = len(X_train_scaled[0])
hidden_nodes_layer1 = 80
hidden_nodes_layer2 = 30
hidden_nodes_layer3 = 10


nn = tf.keras.models.Sequential()

# First hidden layer
nn.add(
    tf.keras.layers.Dense(units=hidden_nodes_layer1, input_dim=number_input_features, activation="relu")
)

# Second hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer2, activation="relu"))

# Third hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer3, activation="relu"))

# Output layer
nn.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Check the structure of the model
nn.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_3 (Dense)              (None, 80)                3520      
_________________________________________________________________
dense_4 (Dense)              (None, 30)                2430      
_________________________________________________________________
dense_5 (Dense)              (None, 10)                310       
_________________________________________________________________
dense_6 (Dense)              (None, 1)                 11        
Total params: 6,271
Trainable params: 6,271
Non-trainable params: 0
_________________________________________________________________


In [17]:
# Compile the model
nn.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

In [18]:
# Import checkpoint dependencies
import os
from tensorflow.keras.callbacks import ModelCheckpoint

# Define the checkpoint path and filenames
os.makedirs("checkpoints/",exist_ok=True)
checkpoint_path = "checkpoints/weights.{epoch:02d}.hdf5"

# Compile the model
nn.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

# Create a callback that saves the model's weights every epoch
cp_callback = ModelCheckpoint(
    filepath=checkpoint_path,
    verbose=1,
    save_weights_only=True,
    save_freq='epoch')

# Train the model
fit_model = nn.fit(X_train_scaled,y_train,epochs=100,callbacks=[cp_callback])

Train on 25724 samples
Epoch 1/100
25664/25724 [============================>.] - ETA: 0s - loss: 0.5690 - accuracy: 0.7231
Epoch 00001: saving model to checkpoints/weights.01.hdf5
25724/25724 [==============================] - 3s 109us/sample - loss: 0.5688 - accuracy: 0.7232
Epoch 2/100
25472/25724 [============================>.] - ETA: 0s - loss: 0.5549 - accuracy: 0.7303
Epoch 00002: saving model to checkpoints/weights.02.hdf5
25724/25724 [==============================] - 2s 89us/sample - loss: 0.5549 - accuracy: 0.7303
Epoch 3/100
25504/25724 [============================>.] - ETA: 0s - loss: 0.5522 - accuracy: 0.7306
Epoch 00003: saving model to checkpoints/weights.03.hdf5
25724/25724 [==============================] - 2s 91us/sample - loss: 0.5524 - accuracy: 0.7307
Epoch 4/100
25568/25724 [============================>.] - ETA: 0s - loss: 0.5507 - accuracy: 0.7322
Epoch 00004: saving model to checkpoints/weights.04.hdf5
25724/25724 [==============================] - 2s 90us/s

25632/25724 [============================>.] - ETA: 0s - loss: 0.5405 - accuracy: 0.7379
Epoch 00033: saving model to checkpoints/weights.33.hdf5
25724/25724 [==============================] - 8s 326us/sample - loss: 0.5404 - accuracy: 0.7380
Epoch 34/100
25600/25724 [============================>.] - ETA: 0s - loss: 0.5400 - accuracy: 0.7385
Epoch 00034: saving model to checkpoints/weights.34.hdf5
25724/25724 [==============================] - 2s 90us/sample - loss: 0.5400 - accuracy: 0.7384
Epoch 35/100
25248/25724 [============================>.] - ETA: 0s - loss: 0.5409 - accuracy: 0.7378
Epoch 00035: saving model to checkpoints/weights.35.hdf5
25724/25724 [==============================] - 2s 90us/sample - loss: 0.5402 - accuracy: 0.7383
Epoch 36/100
25280/25724 [============================>.] - ETA: 0s - loss: 0.5394 - accuracy: 0.7386
Epoch 00036: saving model to checkpoints/weights.36.hdf5
25724/25724 [==============================] - 2s 90us/sample - loss: 0.5398 - accuracy:

25632/25724 [============================>.] - ETA: 0s - loss: 0.5372 - accuracy: 0.7403
Epoch 00065: saving model to checkpoints/weights.65.hdf5
25724/25724 [==============================] - 9s 342us/sample - loss: 0.5373 - accuracy: 0.7404
Epoch 66/100
25632/25724 [============================>.] - ETA: 0s - loss: 0.5371 - accuracy: 0.7405
Epoch 00066: saving model to checkpoints/weights.66.hdf5
25724/25724 [==============================] - 5s 190us/sample - loss: 0.5373 - accuracy: 0.7403
Epoch 67/100
25568/25724 [============================>.] - ETA: 0s - loss: 0.5374 - accuracy: 0.7396
Epoch 00067: saving model to checkpoints/weights.67.hdf5
25724/25724 [==============================] - 9s 339us/sample - loss: 0.5371 - accuracy: 0.7398
Epoch 68/100
25664/25724 [============================>.] - ETA: 0s - loss: 0.5371 - accuracy: 0.7401
Epoch 00068: saving model to checkpoints/weights.68.hdf5
25724/25724 [==============================] - 12s 481us/sample - loss: 0.5371 - accur

25600/25724 [============================>.] - ETA: 0s - loss: 0.5354 - accuracy: 0.7408
Epoch 00097: saving model to checkpoints/weights.97.hdf5
25724/25724 [==============================] - 5s 193us/sample - loss: 0.5359 - accuracy: 0.7406
Epoch 98/100
25696/25724 [============================>.] - ETA: 0s - loss: 0.5360 - accuracy: 0.7406
Epoch 00098: saving model to checkpoints/weights.98.hdf5
25724/25724 [==============================] - 11s 424us/sample - loss: 0.5359 - accuracy: 0.7407
Epoch 99/100
25696/25724 [============================>.] - ETA: 0s - loss: 0.5354 - accuracy: 0.7403
Epoch 00099: saving model to checkpoints/weights.99.hdf5
25724/25724 [==============================] - 19s 734us/sample - loss: 0.5354 - accuracy: 0.7404
Epoch 100/100
25696/25724 [============================>.] - ETA: 0s - loss: 0.5357 - accuracy: 0.7409
Epoch 00100: saving model to checkpoints/weights.100.hdf5
25724/25724 [==============================] - 24s 951us/sample - loss: 0.5357 - a

In [19]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

8575/1 - 1s - loss: 0.5417 - accuracy: 0.7256
Loss: 0.5567895244578926, Accuracy: 0.7255976796150208


### Using Different Activation Functions for the Hidden Layers

In [20]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
number_input_features = len(X_train_scaled[0])
hidden_nodes_layer1 = 80
hidden_nodes_layer2 = 30
hidden_nodes_layer3 = 10


nn = tf.keras.models.Sequential()

# First hidden layer
nn.add(
    tf.keras.layers.Dense(units=hidden_nodes_layer1, input_dim=number_input_features, activation="relu")
)

# Second hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer2, activation="tanh"))

# Third hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer3, activation="tanh"))

# Output layer
nn.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Check the structure of the model
nn.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_7 (Dense)              (None, 80)                3520      
_________________________________________________________________
dense_8 (Dense)              (None, 30)                2430      
_________________________________________________________________
dense_9 (Dense)              (None, 10)                310       
_________________________________________________________________
dense_10 (Dense)             (None, 1)                 11        
Total params: 6,271
Trainable params: 6,271
Non-trainable params: 0
_________________________________________________________________


In [21]:
# Compile the model
nn.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

In [22]:
# Import checkpoint dependencies
import os
from tensorflow.keras.callbacks import ModelCheckpoint

# Define the checkpoint path and filenames
os.makedirs("checkpoints/",exist_ok=True)
checkpoint_path = "checkpoints/weights.{epoch:02d}.hdf5"

# Compile the model
nn.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

# Create a callback that saves the model's weights every epoch
cp_callback = ModelCheckpoint(
    filepath=checkpoint_path,
    verbose=1,
    save_weights_only=True,
    save_freq='epoch')

# Train the model
fit_model = nn.fit(X_train_scaled,y_train,epochs=100,callbacks=[cp_callback])

Train on 25724 samples
Epoch 1/100
25376/25724 [============================>.] - ETA: 0s - loss: 0.5692 - accuracy: 0.7186
Epoch 00001: saving model to checkpoints/weights.01.hdf5
25724/25724 [==============================] - 3s 113us/sample - loss: 0.5687 - accuracy: 0.7190
Epoch 2/100
25536/25724 [============================>.] - ETA: 0s - loss: 0.5554 - accuracy: 0.7273
Epoch 00002: saving model to checkpoints/weights.02.hdf5
25724/25724 [==============================] - 3s 106us/sample - loss: 0.5552 - accuracy: 0.7274
Epoch 3/100
25344/25724 [============================>.] - ETA: 0s - loss: 0.5518 - accuracy: 0.7313
Epoch 00003: saving model to checkpoints/weights.03.hdf5
25724/25724 [==============================] - 2s 90us/sample - loss: 0.5520 - accuracy: 0.7309
Epoch 4/100
25504/25724 [============================>.] - ETA: 0s - loss: 0.5498 - accuracy: 0.7318
Epoch 00004: saving model to checkpoints/weights.04.hdf5
25724/25724 [==============================] - 2s 92us/

Epoch 33/100
25696/25724 [============================>.] - ETA: 0s - loss: 0.5384 - accuracy: 0.7388
Epoch 00033: saving model to checkpoints/weights.33.hdf5
25724/25724 [==============================] - 8s 295us/sample - loss: 0.5384 - accuracy: 0.7387
Epoch 34/100
25440/25724 [============================>.] - ETA: 0s - loss: 0.5384 - accuracy: 0.7387
Epoch 00034: saving model to checkpoints/weights.34.hdf5
25724/25724 [==============================] - 6s 252us/sample - loss: 0.5385 - accuracy: 0.7389
Epoch 35/100
25696/25724 [============================>.] - ETA: 0s - loss: 0.5383 - accuracy: 0.7393
Epoch 00035: saving model to checkpoints/weights.35.hdf5
25724/25724 [==============================] - 9s 347us/sample - loss: 0.5384 - accuracy: 0.7392
Epoch 36/100
25536/25724 [============================>.] - ETA: 0s - loss: 0.5385 - accuracy: 0.7392
Epoch 00036: saving model to checkpoints/weights.36.hdf5
25724/25724 [==============================] - 7s 256us/sample - loss: 0.

Epoch 65/100
25632/25724 [============================>.] - ETA: 0s - loss: 0.5357 - accuracy: 0.7408
Epoch 00065: saving model to checkpoints/weights.65.hdf5
25724/25724 [==============================] - 9s 357us/sample - loss: 0.5359 - accuracy: 0.7407
Epoch 66/100
25312/25724 [============================>.] - ETA: 0s - loss: 0.5363 - accuracy: 0.7399
Epoch 00066: saving model to checkpoints/weights.66.hdf5
25724/25724 [==============================] - 6s 231us/sample - loss: 0.5360 - accuracy: 0.7404
Epoch 67/100
25696/25724 [============================>.] - ETA: 0s - loss: 0.5359 - accuracy: 0.7402
Epoch 00067: saving model to checkpoints/weights.67.hdf5
25724/25724 [==============================] - 8s 306us/sample - loss: 0.5359 - accuracy: 0.7403
Epoch 68/100
25472/25724 [============================>.] - ETA: 0s - loss: 0.5354 - accuracy: 0.7409
Epoch 00068: saving model to checkpoints/weights.68.hdf5
25724/25724 [==============================] - 6s 246us/sample - loss: 0.

Epoch 97/100
25536/25724 [============================>.] - ETA: 0s - loss: 0.5354 - accuracy: 0.7403
Epoch 00097: saving model to checkpoints/weights.97.hdf5
25724/25724 [==============================] - 4s 143us/sample - loss: 0.5349 - accuracy: 0.7410
Epoch 98/100
25440/25724 [============================>.] - ETA: 0s - loss: 0.5353 - accuracy: 0.7405
Epoch 00098: saving model to checkpoints/weights.98.hdf5
25724/25724 [==============================] - 5s 208us/sample - loss: 0.5348 - accuracy: 0.7408
Epoch 99/100
25664/25724 [============================>.] - ETA: 0s - loss: 0.5341 - accuracy: 0.7412
Epoch 00099: saving model to checkpoints/weights.99.hdf5
25724/25724 [==============================] - 5s 203us/sample - loss: 0.5343 - accuracy: 0.7410
Epoch 100/100
25600/25724 [============================>.] - ETA: 0s - loss: 0.5342 - accuracy: 0.7419
Epoch 00100: saving model to checkpoints/weights.100.hdf5
25724/25724 [==============================] - 4s 158us/sample - loss: 

In [23]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

8575/1 - 1s - loss: 0.5554 - accuracy: 0.7254
Loss: 0.5537820277130638, Accuracy: 0.7253644466400146


### Adding or reducing the number of epochs to the training regimen

In [24]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
number_input_features = len(X_train_scaled[0])
hidden_nodes_layer1 = 80
hidden_nodes_layer2 = 30
hidden_nodes_layer3 = 10


nn = tf.keras.models.Sequential()

# First hidden layer
nn.add(
    tf.keras.layers.Dense(units=hidden_nodes_layer1, input_dim=number_input_features, activation="relu")
)

# Second hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer2, activation="relu"))

# Third hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer3, activation="relu"))

# Output layer
nn.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Check the structure of the model
nn.summary()

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_11 (Dense)             (None, 80)                3520      
_________________________________________________________________
dense_12 (Dense)             (None, 30)                2430      
_________________________________________________________________
dense_13 (Dense)             (None, 10)                310       
_________________________________________________________________
dense_14 (Dense)             (None, 1)                 11        
Total params: 6,271
Trainable params: 6,271
Non-trainable params: 0
_________________________________________________________________


In [25]:
# Compile the model
nn.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

In [26]:
# Import checkpoint dependencies
import os
from tensorflow.keras.callbacks import ModelCheckpoint

# Define the checkpoint path and filenames
os.makedirs("checkpoints/",exist_ok=True)
checkpoint_path = "checkpoints/weights.{epoch:02d}.hdf5"

# Compile the model
nn.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

# Create a callback that saves the model's weights every epoch
cp_callback = ModelCheckpoint(
    filepath=checkpoint_path,
    verbose=1,
    save_weights_only=True,
    save_freq='epoch')

# Train the model
fit_model = nn.fit(X_train_scaled,y_train,epochs=200,callbacks=[cp_callback])

Train on 25724 samples
Epoch 1/200
25280/25724 [============================>.] - ETA: 0s - loss: 0.5754 - accuracy: 0.7158
Epoch 00001: saving model to checkpoints/weights.01.hdf5
25724/25724 [==============================] - 3s 109us/sample - loss: 0.5746 - accuracy: 0.7166
Epoch 2/200
25248/25724 [============================>.] - ETA: 0s - loss: 0.5549 - accuracy: 0.7303
Epoch 00002: saving model to checkpoints/weights.02.hdf5
25724/25724 [==============================] - 2s 90us/sample - loss: 0.5548 - accuracy: 0.7304
Epoch 3/200
25568/25724 [============================>.] - ETA: 0s - loss: 0.5525 - accuracy: 0.7319
Epoch 00003: saving model to checkpoints/weights.03.hdf5
25724/25724 [==============================] - 3s 108us/sample - loss: 0.5526 - accuracy: 0.7317
Epoch 4/200
25472/25724 [============================>.] - ETA: 0s - loss: 0.5510 - accuracy: 0.7314
Epoch 00004: saving model to checkpoints/weights.04.hdf5
25724/25724 [==============================] - 2s 91us/

25536/25724 [============================>.] - ETA: 0s - loss: 0.5408 - accuracy: 0.7385
Epoch 00033: saving model to checkpoints/weights.33.hdf5
25724/25724 [==============================] - 2s 95us/sample - loss: 0.5405 - accuracy: 0.7390
Epoch 34/200
25248/25724 [============================>.] - ETA: 0s - loss: 0.5410 - accuracy: 0.7371
Epoch 00034: saving model to checkpoints/weights.34.hdf5
25724/25724 [==============================] - 2s 96us/sample - loss: 0.5404 - accuracy: 0.7377
Epoch 35/200
25376/25724 [============================>.] - ETA: 0s - loss: 0.5409 - accuracy: 0.7382
Epoch 00035: saving model to checkpoints/weights.35.hdf5
25724/25724 [==============================] - 3s 109us/sample - loss: 0.5403 - accuracy: 0.7388
Epoch 36/200
25312/25724 [============================>.] - ETA: 0s - loss: 0.5402 - accuracy: 0.7378
Epoch 00036: saving model to checkpoints/weights.36.hdf5
25724/25724 [==============================] - 3s 122us/sample - loss: 0.5399 - accuracy

25184/25724 [============================>.] - ETA: 0s - loss: 0.5374 - accuracy: 0.7399
Epoch 00065: saving model to checkpoints/weights.65.hdf5
25724/25724 [==============================] - 3s 101us/sample - loss: 0.5377 - accuracy: 0.7399
Epoch 66/200
25376/25724 [============================>.] - ETA: 0s - loss: 0.5368 - accuracy: 0.7400
Epoch 00066: saving model to checkpoints/weights.66.hdf5
25724/25724 [==============================] - 2s 94us/sample - loss: 0.5369 - accuracy: 0.7399
Epoch 67/200
25664/25724 [============================>.] - ETA: 0s - loss: 0.5378 - accuracy: 0.7401
Epoch 00067: saving model to checkpoints/weights.67.hdf5
25724/25724 [==============================] - 3s 97us/sample - loss: 0.5379 - accuracy: 0.7400
Epoch 68/200
25344/25724 [============================>.] - ETA: 0s - loss: 0.5370 - accuracy: 0.7403
Epoch 00068: saving model to checkpoints/weights.68.hdf5
25724/25724 [==============================] - 3s 97us/sample - loss: 0.5370 - accuracy:

25440/25724 [============================>.] - ETA: 0s - loss: 0.5366 - accuracy: 0.7397
Epoch 00097: saving model to checkpoints/weights.97.hdf5
25724/25724 [==============================] - 5s 192us/sample - loss: 0.5361 - accuracy: 0.7402
Epoch 98/200
25600/25724 [============================>.] - ETA: 0s - loss: 0.5356 - accuracy: 0.7406
Epoch 00098: saving model to checkpoints/weights.98.hdf5
25724/25724 [==============================] - 2s 97us/sample - loss: 0.5359 - accuracy: 0.7403
Epoch 99/200
25600/25724 [============================>.] - ETA: 0s - loss: 0.5362 - accuracy: 0.7405
Epoch 00099: saving model to checkpoints/weights.99.hdf5
25724/25724 [==============================] - 3s 129us/sample - loss: 0.5359 - accuracy: 0.7407
Epoch 100/200
25504/25724 [============================>.] - ETA: 0s - loss: 0.5365 - accuracy: 0.7402
Epoch 00100: saving model to checkpoints/weights.100.hdf5
25724/25724 [==============================] - 4s 169us/sample - loss: 0.5360 - accur

Epoch 129/200
25440/25724 [============================>.] - ETA: 0s - loss: 0.5344 - accuracy: 0.7411
Epoch 00129: saving model to checkpoints/weights.129.hdf5
25724/25724 [==============================] - 3s 108us/sample - loss: 0.5344 - accuracy: 0.7412
Epoch 130/200
25408/25724 [============================>.] - ETA: 0s - loss: 0.5351 - accuracy: 0.7402
Epoch 00130: saving model to checkpoints/weights.130.hdf5
25724/25724 [==============================] - 3s 122us/sample - loss: 0.5349 - accuracy: 0.7407
Epoch 131/200
25440/25724 [============================>.] - ETA: 0s - loss: 0.5345 - accuracy: 0.7406
Epoch 00131: saving model to checkpoints/weights.131.hdf5
25724/25724 [==============================] - 4s 140us/sample - loss: 0.5345 - accuracy: 0.7407
Epoch 132/200
25440/25724 [============================>.] - ETA: 0s - loss: 0.5346 - accuracy: 0.7413
Epoch 00132: saving model to checkpoints/weights.132.hdf5
25724/25724 [==============================] - 3s 98us/sample - l

Epoch 161/200
25376/25724 [============================>.] - ETA: 0s - loss: 0.5337 - accuracy: 0.7420
Epoch 00161: saving model to checkpoints/weights.161.hdf5
25724/25724 [==============================] - 3s 125us/sample - loss: 0.5340 - accuracy: 0.7419
Epoch 162/200
25632/25724 [============================>.] - ETA: 0s - loss: 0.5340 - accuracy: 0.7409
Epoch 00162: saving model to checkpoints/weights.162.hdf5
25724/25724 [==============================] - 2s 95us/sample - loss: 0.5338 - accuracy: 0.7411
Epoch 163/200
25696/25724 [============================>.] - ETA: 0s - loss: 0.5339 - accuracy: 0.7420
Epoch 00163: saving model to checkpoints/weights.163.hdf5
25724/25724 [==============================] - 3s 101us/sample - loss: 0.5339 - accuracy: 0.7420
Epoch 164/200
25504/25724 [============================>.] - ETA: 0s - loss: 0.5333 - accuracy: 0.7419
Epoch 00164: saving model to checkpoints/weights.164.hdf5
25724/25724 [==============================] - 3s 100us/sample - l

Epoch 193/200
25536/25724 [============================>.] - ETA: 0s - loss: 0.5323 - accuracy: 0.7426
Epoch 00193: saving model to checkpoints/weights.193.hdf5
25724/25724 [==============================] - 2s 96us/sample - loss: 0.5326 - accuracy: 0.7424
Epoch 194/200
25568/25724 [============================>.] - ETA: 0s - loss: 0.5328 - accuracy: 0.7423
Epoch 00194: saving model to checkpoints/weights.194.hdf5
25724/25724 [==============================] - 3s 101us/sample - loss: 0.5330 - accuracy: 0.7422
Epoch 195/200
25472/25724 [============================>.] - ETA: 0s - loss: 0.5330 - accuracy: 0.7421
Epoch 00195: saving model to checkpoints/weights.195.hdf5
25724/25724 [==============================] - 5s 192us/sample - loss: 0.5330 - accuracy: 0.7423
Epoch 196/200
25600/25724 [============================>.] - ETA: 0s - loss: 0.5330 - accuracy: 0.7420
Epoch 00196: saving model to checkpoints/weights.196.hdf5
25724/25724 [==============================] - 4s 144us/sample - l

In [27]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

8575/1 - 1s - loss: 0.5501 - accuracy: 0.7251
Loss: 0.5660576323934616, Accuracy: 0.7251312136650085


In [29]:
# Export our model to HDF5 file
nn.save("AlphabetSoupCharity_Optimization.h5")